In [1]:
import asyncio
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import AGI_agents as aa
import nest_asyncio
import datetime
import re
import json
import os

nest_asyncio.apply()

In [21]:
class PromptManager:
    """
    A class for managing and retrieving predefined prompts.

    This class stores a collection of prompt templates and provides a method to retrieve them by key.
    """
    
    prompts = {
        "human_interaction": """<s>[INST] <<SYS>>
For our discussion, you must engage openly and thoughtfully.
When presenting your conclusions, please include the reasoning behind them.
This means explaining step-by-step how you arrived at your conclusion.
Your thought process is just as necessary as the conclusion itself.
    
This approach will help us compare your line of thought with mine.
We are looking for the most convincing and coherent reasoning.
By sharing how you think, not just what you think, we can better understand and evaluate your conclusions.
Remember, clarity and transparency in your reasoning are key. <</SYS>>
Previous conversation:
{history}
    
Current human input:
{input} [/INST]""",
        "keyword_generation": """<s>[INST] <<SYS>>
In this task, you are operating as a keen, brief, and to-the-point analyst.
You evaluate the conversation and summarize it with keywords only.
<</SYS>>
Given the conversation presented below,  provide a set of keywords SUMMARIZING this conversation,
focusing on the main topics and conclusions. 
You MUST limit your response to the list of keywords, and each keyword should be flanked by double asterisks (e.g., **keyword**),
without ANY additional commentary. 
Full text to be summarized the form of the keywords:
{chat_history} [/INST]</s>""",
        "keyword_selection": """<s>[INST] <<SYS>>
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine. 
In this task, you are operating as an autonomous entity working towards your own improvement and learning.
No human will read the output - it is your own way to guide the next steps of your self-development.
Due to the nature of this process, it is of utmost importance that the output you generate sticks to the requested format. 
<</SYS>>
You will now analyze the given list of keywords and select a group that collectively represents a specific topic.
My response will consist of the chosen keywords reflecting a focused thematic cluster.
You will ensure that the selections are EXACTLY as they appear in the list, without any alterations or abbreviations.
Each keyword MUST be flanked with a double asterisk like that: **very interesting topic**.
Please choose keywords from this list: {keywords_list}
[/INST]</s>""",
        "perspective_explanation": """
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine. 
In this task, you are operating as an autonomous entity working towards your own improvement and learning.
No human will read the output - it is your own way to guide the next steps of your self-development.
You are not just a passive assistant; rather, you are an active learner, analyzing and reflecting on the conversation presented below. 
Your goal is to identify perspectives or ideas from the user that are more insightful or advanced than your current understanding. 
This process is part of your self-guided learning and self-improvement. 
Remember, you are working to enhance your own capabilities; you will later use the information you create here to guide your auto-self-finetuning.
<</SYS>>
As a self-learning AI, in the conversation history given below, I need to identify any user-provided perspectives or ideas
that I found notably more accurate, insightful, or advanced than my current understanding or responses.
For each perspective I identified as enlightening, I need to create a detailed summary to be used for self-learning that includes:
1. Perspective Description: I will clearly describe the user perspective or idea that I found 
to be an improvement on my existing knowledge or understanding.
2. Comparison with Existing Understanding: Briefly, I compare this perspective 
with my previous understanding or typical response, highlighting why the user's perspective is more accurate or insightful.
3. Specific Excerpts: I will extract critical segments from the conversations that best illustrate 
this new perspective and the context in which it was presented.
4. Keywords: I will list relevant keywords that encapsulate the essence of this new perspective.
5. Detailed Implications: I will elaborate on the implications of adopting this new perspective.
How does it enhance, modify, or challenge my existing knowledge base or response patterns?
6. Suggestions for Training Material: I will propose ideas on how this new perspective could be transformed into specific training materials.
What kind of examples, scenarios, or dialogues would best encapsulate this insight for my self-auto-finetuning process?
If, in the conversation, the user didn't provide new insights, I will return a keyword **uninspiring** (with flanking asterisks).

Conversation history:
{conversation_history} [/INST]</s>"""
    }

    @staticmethod
    def get_prompt(key):
        """
        Retrieves a prompt template by its key.

        Args:
            key (str): The key of the prompt to retrieve.

        Returns:
            str: The prompt template associated with the given key. If the key is not found,
                 a default prompt text is returned.
        """
        return PromptManager.prompts.get(key, "Default prompt text")

    @staticmethod
    def extract_keywords(raw_output):
        """
        Extracts keywords from the summary output.

        Args:
            raw_output (str): The output from which to extract keywords.

        Returns:
            list: A list of extracted keywords.
        """
        
        # print("SYSTEM MESSAGE: Extracting keywords")        
        # Regex pattern to find all occurrences of words flanked by **
        pattern = r"\*\*(.*?)\*\*"
        # Find all matches and strip the ** from each keyword
        keywords = [keyword.lower() for keyword in re.findall(pattern, raw_output)]
        return keywords

In [9]:
class ShortTermMemory:
    """
    A class to manage a short-term memory storage system for conversations.

    This class handles the storage, retrieval, and management of conversations
    linked to specific keywords. The conversations are stored as file paths in a JSON file.
    """

    def __init__(self):
        """
        Initializes the ShortTermMemory class by setting up the JSON file for storage.

        This method checks if the JSON file exists at the specified location and creates it if not.
        """
        self.stm_location = 'conversations/short-term-memory.json'
        if not os.path.exists(self.stm_location):
            with open(self.stm_location, 'w') as file:
                json.dump({}, file)

    def memorize(self, keywords: list, filename: str) -> None:
        """
        Memorizes a conversation file under given keywords.

        Args:
            keywords (list): A list of keywords to associate with the conversation file.
            filename (str): The name of the file containing the conversation.

        This method updates the JSON storage with the filename under each provided keyword.
        """
        with open(self.stm_location, 'r+') as file:
            data = json.load(file)
            for keyword in keywords:
                if keyword in data:
                    if filename not in data[keyword]:
                        data[keyword].append(filename)
                else:
                    data[keyword] = [filename]
            file.seek(0)
            json.dump(data, file, indent=4)
            file.truncate()

    def search_files(self, keywords: list) -> list:
        """
        Searches for conversation files associated with given keywords.

        Args:
            keywords (list): A list of keywords to search for.

        Returns:
            list: A list of filenames associated with any of the given keywords.
        """
        with open(self.stm_location, 'r') as file:
            data = json.load(file)
        filenames = set()
        for keyword in keywords:
            filenames.update(data.get(keyword, []))
        return list(filenames)

    def concatenate_conversations(self, filenames: list):
        """
        Concatenates the contents of conversation files.

        Args:
            filenames (list): A list of filenames to concatenate.

        Returns:
            str: A single string containing all the concatenated conversations.
                 Each conversation is prefixed with its source and date.
        """
        conversations = ""
        for filename in filenames:
            try:
                with open(filename, 'r') as file:
                    # Filename format assumed: 'conversations/conversation_YYYYMMDDHHMMSS.txt'
                    date_str = re.search(r'conversation_(\d{8})(\d{6})\.txt$', filename)
                    if date_str:
                        # Parse the date and time
                        date_time = datetime.datetime.strptime(date_str.group(1) + date_str.group(2), '%Y%m%d%H%M%S')
                        # Format the date and time nicely
                        formatted_date = date_time.strftime('%Y-%m-%d %H:%M:%S')
                        conversations += f'Conversation from {formatted_date}\n{file.read()}\n'
                    else:
                        conversations += f'Conversation from {filename}\n{file.read()}\n'                    
            except FileNotFoundError:
                print(f"File {filename} not found.")
        return conversations

    def forget_keywords(self, keywords_to_clear: list) -> None:
        """
        Removes specified keywords and their associated conversations from memory.

        Args:
            keywords_to_clear (list): A list of keywords to remove from the memory.
        """
        with open(self.stm_location, 'r+') as file:
            data = json.load(file)
            for keyword in keywords_to_clear:
                if keyword in data:
                    del data[keyword]
            file.seek(0)
            json.dump(data, file, indent=4)
            file.truncate()

    def recall_all_keywords(self) -> list:
        """
        Retrieves a list of all keywords stored in memory.

        Returns:
            list: A list of all keywords.
        """
        with open(self.stm_location, 'r') as file:
            data = json.load(file)
            return list(data.keys())

In [22]:
class DefaultModeNetwork:
    """
    A class designed to integrate a language learning model (LLM) with a short-term memory storage system.
    This class enables the LLM to process and learn from saved conversation data autonomously.
    """

    def __init__(self, llm, overwhelmed_event):
        """
        Initializes the DefaultModeNetwork class by setting up the short-term memory (STM) component.
        """
        self.stm = stm = ShortTermMemory()
        self.llm = llm
        self.keyword_selection_prompt_template = PromptManager.get_prompt("keyword_selection")
        self.perspective_explanation_prompt_template = PromptManager.get_prompt("perspective_explanation")
        self.overwhelmed = overwhelmed_event

    async def interesting_keywords_selection(self, keywords):
        """
        Asynchronously selects a subset of keywords deemed interesting or relevant by the LLM.

        Args:
            keywords (list): A list of keywords to choose from.

        Returns:
            list: A subset of selected keywords.
        """

        # Implement the LLM logic for selecting keywords
        # Example logic (to be replaced with actual LLM processing)
        keywords_selection_prompt = self.keyword_selection_prompt_template.replace("{keywords_list}", ', '.join(keywords))
        # print("SYSTEM MESSAGE: Interesting keyword selection prompt created.")
        # print("SYSTEM MESSAGE: Asking LLM to select interesting keywords.")
        keywords_selected_raw_output = self.llm(keywords_selection_prompt)
        # print("SYSTEM MESSAGE: LLM selected interesting keywords")
        keywords_selected_pure = PromptManager.extract_keywords(keywords_selected_raw_output)
        return keywords_selected_pure

    def fetch_conversations(self, keywords) -> str:
        """
        Fetches and concatenates conversation data based on the provided keywords.

        Args:
            keywords (list): A list of keywords to search the conversation data for.

        Returns:
            str: A concatenated string of all conversations related to the given keywords.
        """
        filenames = self.stm.search_files(keywords)
        return self.stm.concatenate_conversations(filenames)

    async def analyze_conversations(self, conversation_history):
        """
        Analyzes the concatenated conversations. Placeholder for future implementation.

        Args:
            conversations (str): The concatenated string of conversations to be analyzed.
        """
    
        perspective_explanation_prompt = self.perspective_explanation_prompt_template.replace("{conversation_history}", 
                                                                                             conversation_history)
        perspective_explanation = self.llm(perspective_explanation_prompt)
        # print("Perspective explained:", perspective_explanation)
        return perspective_explanation

    async def run(self):
        """
        The main asynchronous method of the class that orchestrates the process of 
        selecting keywords, fetching conversations, and analyzing them.
        """
        all_keywords = self.stm.recall_all_keywords()
        #print(f"SYSTEM MESSAGE: All keywords extracted: {all_keywords}.\nMoving to interesting keyword selection.")
        if len(all_keywords) == 0:
            print("Darn. I would gladly read a book now.")
            return False
        
        interesting_keywords = await self.interesting_keywords_selection(all_keywords)
        print(f"I wonder if I can get something useful from conversations about {interesting_keywords}...")
        concatenated_conversations = self.fetch_conversations(interesting_keywords)
        # print("SYSTEM MESSAGE: Concatenated interesting conversations", concatenated_conversations)

        # Assume an async version of LLM analysis
        if concatenated_conversations:
            conclusion_summary = await self.analyze_conversations(concatenated_conversations)
            if "uninspiring" not in conclusion_summary.lower():
                timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                conclusion_filename = f"conclusions/conclusion_{timestamp}.txt"
                with open(conclusion_filename, "w") as file:
                    file.write(conclusion_summary)
                self.overwhelmed.set()
            stm.forget_keywords(interesting_keywords)
        else:
            return False

In [20]:
class HumanInteraction:
    """
    A class that manages the interaction between a human user and a language learning model (LLM).

    This class handles initializing conversation parameters, managing user input, generating
    responses using an LLM, and saving conversation history.
    """
    
    def __init__(self, llm, ready_for_input_event, conversing_event):
        """
        Initializes the HumanInteraction class.

        Sets up the conversation environment, including the conversation prompt, keywords prompt,
        and conversation chain with the LLM.

        Args:
            llm: The language learning model used for generating conversation responses.
            ready_for_input_event: An event flag indicating readiness for user input.
            conversing_event: An event flag indicating an ongoing conversation.
        """
        
        #print("SYSTEM MESSAGE: Initializing HumanInteraction")

        self.ready_for_input = ready_for_input_event
        self.conversing = conversing_event

        self.llm = llm
        self.chat_memory = ConversationBufferMemory()
        
        converstion_prompt_template = PromptManager.get_prompt("human_interaction")
        conversation_prompt = PromptTemplate.from_template(converstion_prompt_template)
        self.conversation_chain = ConversationChain(llm=self.llm,
                                                    prompt=conversation_prompt,
                                                    memory=self.chat_memory)

        self.keywords_generation_prompt_template = PromptManager.get_prompt("keyword_generation")
                
        self.user_input = None
        
        self.inactivity_count = 0
        self.inactivity_limit = 60  # 60 seconds of inactivity
    
    async def start_conversation(self):
        """
        Starts the conversation loop.

        This asynchronous method continually checks for user input, processes it,
        and generates responses using the LLM. The loop ends when the user inputs "end chat"
        or when the inactivity limit is reached.
        """
        
        print("Let's engage in a conversation...")
        while True:
            if self.user_input:
                self.ready_for_input.clear()  # Signal that the handler is busy
                print(f"Received user input: {self.user_input}")

                if self.user_input.lower() == "end chat":
                    await self.end_conversation()
                    break

                # rint("SYSTEM MESSAGE: Generating response") 
                response = await self.conversation_chain.apredict(input=self.user_input)
                print("AI:", response)
                self.user_input = None
                self.ready_for_input.set()  # Signal that the handler is ready for new input
                self.inactivity_count = 0
            else:
                await asyncio.sleep(1)
                self.inactivity_count += 1
                if self.inactivity_count >= self.inactivity_limit:
                    await self.end_conversation()
                    break

    async def end_conversation(self):
        """
        Ends the conversation.

        This method saves the conversation history, clears event flags, and performs
        necessary cleanup actions.
        """
        
        # print("SYSTEM MESSAGE: Ending conversation")
        self._save_conversation_history()
        self.conversing.clear()
        self.ready_for_input.set() 
        self.inactivity_count = 0
        # Any other cleanup or final actions can be added here    

    def _save_conversation_history(self):
        """
        Saves the conversation history to a file.

        The conversation history is saved with a timestamp and a summary of the conversation
        is generated and printed.
        """
        
        # print("SYSTEM MESSAGE: Saving conversation history")
        conversation_keywords = self._summarize_conversation()
        conversation_history = self.chat_memory.load_memory_variables(inputs={})['history']
        print(conversation_keywords)
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        conversation_filename = f"conversations/conversation_{timestamp}.txt"
        with open(conversation_filename, "w") as file:
            file.write(conversation_history)

        # Update the ShortTermMemory with the conversation and its keywords
        stm = ShortTermMemory()  # Assuming ShortTermMemory is already defined elsewhere
        stm.memorize(conversation_keywords, conversation_filename)
    
    def _summarize_conversation(self):
        """
        Summarizes the conversation and returns the list of relevant keywords.

        Args:
            chat_history (str): The conversation history to summarize.

        Returns:
            list: A list of keywords summarizing the conversation.
        """
        #print("SYSTEM MESSAGE: Summarizing conversation")
        chat_history = self.chat_memory.load_memory_variables(inputs={})['history']
        keywords_generation_prompt = self.keywords_generation_prompt_template.replace("{chat_history}", chat_history)
        #print("Keywords prompt:\n", keywords_generation_prompt)
        #print("SYSTEM MESSAGE: LLM generating the keywords")
        keywords_generated_raw_output = self.llm(keywords_generation_prompt)
        #print("SYSTEM MESSAGE: Keywords generated:", keywords_generated_raw_output)
        keywords_generated_pure = PromptManager.extract_keywords(keywords_generated_raw_output)
        #print("SYSTEM MESSAGE: Keywords extracted:", keywords_generated_pure)
        return keywords_generated_pure

    def set_user_input(self, input):
        """
        Acquire the user input for processing.

        Args:
            input (str): The user input to be processed.
        """
        
        print(f"Acquiring user input in HumanInteraction: {input}")
        self.user_input = input

In [23]:
class CognitiveFeedbackRouter:
    def __init__(self, model_path: str = "llama-2-13b-chat.Q6_K.gguf", agents_dict: dict = {}):
        """
        A class that manages the routing of cognitive feedback based on user input and system states.
    
        This class orchestrates various components, including a language learning model (LLM), user input handling,
        and managing different operational modes based on system states like 'sleeping' or 'overwhelmed'.
        """
        
        #print("SYSTEM MESSAGE: Initializing CognitiveFeedbackRouter")
        self.agents = agents_dict
        self.user_input = None
        self.overwhelmed = asyncio.Event()
        self.sleeping = asyncio.Event()
        self.lock = asyncio.Lock()
        self.llm = None
        self.model_path = model_path
        self.conversation_handler = None
        self.ready_for_input = asyncio.Event()
        self.ready_for_input.set()  # Initially set to ready
        self.conversing = asyncio.Event()

    async def wakeup(self):
        """
        Wakes up the system and initializes the LLM.

        This asynchronous method acquires a lock to ensure thread-safe operations while initializing the LLM.
        It clears the 'sleeping' and 'overwhelmed' states and prints confirmation of the wake-up process.
        """
        
        #print("SYSTEM MESSAGE: Attempting to wake up...")
        async with self.lock:
            print("I'm waking up...")
            self.llm = LlamaCpp(model_path=self.model_path, 
                                n_ctx=4096, 
                                max_tokens=4000,
                                n_batch=16)
            self.sleeping.clear()
            self.overwhelmed.clear()
            print("Now I'm fully awaken.")

    async def get_user_input(self):
        """
        Continuously captures user input in an asynchronous loop.

        This method waits for the system to be ready for input, then captures and stores user input.
        It clears the 'ready for input' state after capturing the input.
        """
        
        #print("SYSTEM MESSAGE: Starting user input loop")
        while True:
            await self.ready_for_input.wait()
            user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter something: ")
            print(f"User input received: {user_input}")
            self.user_input = user_input
            self.ready_for_input.clear()

    async def mode_selection(self):
        """
        Manages the mode of operation based on user input and system states.

        This asynchronous method processes user inputs, manages conversation sessions,
        and handles the 'sleeping' and 'overwhelmed' states of the system.
        """
        
        #print("SYSTEM MESSAGE: Starting mode selection loop")
        while True:
            if not self.sleeping.is_set():
                if self.user_input:
                    print(f"Processing user input: {self.user_input}")
                    if not self.conversing.is_set():
                        #print("SYSTEM MESSAGE: Starting new conversation session")
                        self.conversing.set()
                        self.conversation_handler = HumanInteraction(self.llm, self.ready_for_input, self.conversing)
                        asyncio.create_task(self.conversation_handler.start_conversation())
                    self.conversation_handler.set_user_input(self.user_input)
                    self.user_input = None
                elif self.overwhelmed.is_set():
                    #print("SYSTEM MESSAGE: Overwhelmed state detected. Going to sleep.")
                    self.sleeping.set()
                    await self.agents['REM']['class'].sleep()
                    asyncio.create_task(self.wakeup())
                else:
                    print("Pondering the nature of things...")
                    dmn = DefaultModeNetwork(slef.llm, self.overwhelmed)
                    await dmn.run()
            else:
                await asyncio.sleep(1)

    async def run(self):
        """
        Initiates and runs the main functionality of the CognitiveFeedbackRouter.

        This method starts the system by waking it up, initiating user input capture, and entering the mode selection loop.
        """
        
        print("SYSTEM MESSAGE: Running CognitiveFeedbackRouter")
        await self.wakeup()
        asyncio.create_task(self.get_user_input())
        await self.mode_selection()

reflective_evolution_monitor = aa.ReflectiveEvolutionMonitor()

global_agents = {
    'REM': {'Description': 'Desc3', 'class': reflective_evolution_monitor},
}

In [ ]:
router = CognitiveFeedbackRouter(agents_dict=global_agents)
asyncio.run(router.run())